In [0]:
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://transformtest1@justforgen2.dfs.core.windows.net/",
  mount_point = "/mnt/transformtest1",
  extra_configs = configs)

Out[1]: True

In [0]:
dbutils.fs.ls('/mnt/transformtest1/bronze')

Out[4]: [FileInfo(path='dbfs:/mnt/transformtest1/bronze/0fac9eb3-60e0-4a52-8255-4e5382695c82.parquet', name='0fac9eb3-60e0-4a52-8255-4e5382695c82.parquet', size=226201, modificationTime=1699301690000),
 FileInfo(path='dbfs:/mnt/transformtest1/bronze/3905572e-6b4d-450f-91ab-2bba90968679.parquet', name='3905572e-6b4d-450f-91ab-2bba90968679.parquet', size=226201, modificationTime=1699302609000),
 FileInfo(path='dbfs:/mnt/transformtest1/bronze/3d1285f4-888a-4b9a-a168-2174e47d9ea8.parquet', name='3d1285f4-888a-4b9a-a168-2174e47d9ea8.parquet', size=1823, modificationTime=1699302606000),
 FileInfo(path='dbfs:/mnt/transformtest1/bronze/6f79888a-e298-4be8-999c-190c181a384d.parquet', name='6f79888a-e298-4be8-999c-190c181a384d.parquet', size=10116, modificationTime=1699302606000),
 FileInfo(path='dbfs:/mnt/transformtest1/bronze/85783b57-e9ef-4791-8a33-24e5551af0a6.parquet', name='85783b57-e9ef-4791-8a33-24e5551af0a6.parquet', size=9313, modificationTime=1699302606000)]

In [0]:
# Read the existing parquet file for the flights database that was created earlier
flight_df = spark.read.parquet("/mnt/transformtest1/bronze/3d1285f4-888a-4b9a-a168-2174e47d9ea8.parquet")

# Print the schema of the dataframe
flight_df.printSchema()

# Print the flight database size
print("Number of flights in the database: ", flight_df.count())

# Show the first 25 rows (20 is the default)
# To show the first n rows, run: df.show(n)
# The second parameter indicates that column lengths shouldn't be truncated (default is 20 characters)
flight_df.show(25, False)

# # You can also use the DataFrame to run simple queries. Results are returned in a DataFrame.
# # Show the first 25 rows of the results of a query that returns selected colums for all flights originating from airports in Texas
# flight_df.select("TeamName", "Country", "Discipline").filter("Country='Norway'").show(1,False)

# # # Use display to run visualizations
# # # Preferably run this in a separate cmd cell
# display(flight_df)

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)

Number of flights in the database:  46
+---------------------+------+----+-----+
|Discipline           |Female|Male|Total|
+---------------------+------+----+-----+
|3x3 Basketball       |32    |32  |64   |
|Archery              |64    |64  |128  |
|Artistic Gymnastics  |98    |98  |196  |
|Artistic Swimming    |105   |0   |105  |
|Athletics            |969   |1072|2041 |
|Badminton            |86    |87  |173  |
|Baseball/Softball    |90    |144 |234  |
|Basketball           |144   |144 |288  |
|Beach Volleyball     |48    |48  |96   |
|Boxing               |102   |187 |289  |
|Canoe Slalom         |41    |41  |82   |
|Canoe Sprint         |123   |126 |249  |
|Cycling BMX Freestyle|10    |9   |19   |
|Cycling BMX Racing   |24    |24  |48   |
|Cycling Mountain Bike|38    |38  |76   |
|Cycling Road         |70    |131 |201  |
|Cyc

In [0]:
# create a temporary sql view for querying flight information
atheletes_data = spark.read.parquet('/mnt/transformtest1/bronze/athletes.parquet')
atheletes_data.createOrReplaceTempView('AtheletesTable')

# Print the total number of flights in Jan 2016 (the number of rows in the flight data).
# print("Number of flights in Jan 2016: ", atheletes_data.count())
# atheletes_data.show(25, False)

# # Using spark sql, query the parquet file to return the total athletes of each country
num_flights_by_airline=spark.sql("SELECT Country, count(*) AS num FROM AtheletesTable GROUP BY Country ORDER BY num desc")
num_flights_by_airline.show()

# List out all the athletes in Spain
athletes_in_spain = spark.sql(
    "SELECT DISTINCT(PersonName) FROM AtheletesTable WHERE Country = 'Spain'")
print('athletes in Spain: ', athletes_in_spain.count())
# athletes_in_spain.show(100, False)




+--------------------+---+
|             Country|num|
+--------------------+---+
|United States of ...|615|
|               Japan|586|
|           Australia|470|
|People's Republic...|401|
|             Germany|400|
|              France|377|
|              Canada|368|
|       Great Britain|366|
|               Italy|356|
|               Spain|324|
|                 ROC|318|
|              Brazil|291|
|         Netherlands|274|
|   Republic of Korea|223|
|         New Zealand|202|
|              Poland|195|
|           Argentina|180|
|        South Africa|171|
|              Mexico|155|
|             Hungary|155|
+--------------------+---+
only showing top 20 rows

athletes in Spain:  324
